In [1]:
import requests
import json
from bs4 import BeautifulSoup
import re
import time
import csv
import os
import string

def clean_text(text):
    # 한글, 영어, 숫자, 기본적인 문장 부호 및 연산 기호만 남기고 나머지 제거
    return re.sub(r'[^ㄱ-ㅣ가-힣a-zA-Z0-9\s\-\+\=\!\?\,\.\:\;\/\(\)\[\]\{\}\&\*\"\'\~\#\<\>\@]', '', text)


def convert_time(total_time):
    # PT 제거 및 H, M 변환
    match = re.match(r"PT(?:(\d+)H)?(?:(\d+)M)?", total_time)
    if match:
        hours = int(match.group(1) or 0)
        minutes = int(match.group(2) or 0)
        return f"{hours * 60 + minutes} minutes"
    return ""

def extract_yield(recipe_yield):
    # 숫자만 추출
    match = re.search(r"(\d+)", recipe_yield)
    return match.group(1) if match else ""

In [2]:
#페이지의 목록을 가져오는 함수
def fetch_recipes(patterns, output_file):
    all_recipe_ids = set()

    for pattern in patterns:
        page = 1
        while True:
            base_url = f"https://m.10000recipe.com/recipe/list.html?q={pattern}&order=reco&page={page}"
            attempt = 0
            while attempt < 5:
                try:
                    response = requests.get(base_url, timeout=10)
                    print(f"{page}페이지 요청: {pattern}")

                    if response.status_code == 200:
                        soup = BeautifulSoup(response.text, 'html.parser')

                        # "검색결과가 없습니다." 텍스트 확인
                        no_result_text = soup.find(string="검색결과가 없습니다.")
                        if no_result_text:
                            print(f"{pattern}의 {page}페이지에서 더 이상 검색결과가 없습니다.")
                            break

                        # 레시피 목록 추출
                        divs = soup.find_all('div', {'class': 'media'})
                        if not divs:
                            print(f"{pattern}의 {page}페이지에서 더 이상 레시피가 없습니다.")
                            break

                        for div in divs:
                            onclick = div.get('onclick', '')
                            match = re.search(r"/recipe/(\d+)", onclick)
                            if match:
                                recipe_id = match.group(1)
                                all_recipe_ids.add(recipe_id)

                        page += 1  # 다음 페이지로 이동
                        break  # 성공 시 재시도 루프 종료

                    else:
                        print(f"{pattern}의 {page}페이지 요청에 실패했습니다.")
                        break

                except requests.RequestException as e:
                    print(f"Error occurred while requesting {base_url}: {e}")
                    attempt += 1
                    if attempt < 5:
                        print(f"Retrying in 5 minutes... Attempt {attempt}/5")
                        time.sleep(300)
                    else:
                        print("Max retries reached, skipping this page.")
                        break

            if no_result_text:  # 결과 없음이 확인되면 패턴 종료
                break

    # 파일이 없으면 자동 생성하여 저장
    try:
        with open(output_file, 'w', encoding='utf-8') as f:
            for recipe_id in sorted(all_recipe_ids):
                f.write(recipe_id + '\n')
        print(f"총 {len(all_recipe_ids)}개의 레시피 ID가 {output_file}에 저장되었습니다.")
    except Exception as e:
        print(f"파일 저장 중 에러가 발생했습니다: {e}")


In [3]:
def recipecsv(data_file, output_csv_prefix, file_index):
    processed_ids = set()
    current_file_index = file_index
    current_count = 0
    output_csv = f"{output_csv_prefix}_{current_file_index}.csv"

    # 기존 데이터 로드
    if os.path.exists(output_csv):
        with open(output_csv, mode='r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            processed_ids = {row['id'] for row in reader}
            current_count = sum(1 for _ in reader)

    with open(data_file, 'r') as f:
        recipe_ids = [line.strip() for line in f]

    # 레시피 ID를 순회하며 처리
    remaining_ids = []
    for recipe_id in recipe_ids:
        if recipe_id in processed_ids:
            print(f"이미 처리된 레시피 ID: {recipe_id}")
            continue

        url = f"https://m.10000recipe.com/recipe/{recipe_id}"
        attempt = 0

        while attempt < 5:
            try:
                response = requests.get(url, timeout=10)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, 'html.parser')
                    script_tag = soup.find('script', type='application/ld+json')
                    if script_tag:
                        raw_json = script_tag.string
                        fixed_json = re.sub(r'\}\s*\{', '},{', raw_json)

                        try:
                            data = json.loads(fixed_json)

                            # 데이터 추출 및 클리닝
                            name = clean_text(data.get('name', ''))
                            description = clean_text(data.get('description', ''))
                            image = data.get('image', '')
                            total_time = convert_time(data.get('totalTime', ''))
                            recipe_yield = extract_yield(data.get('recipeYield', ''))
                            ingredients = clean_text(','.join(data.get('recipeIngredient', [])))
                            recipe_steps = [clean_text(step.get('text', '')) for step in data.get('recipeInstructions', [])]

                            if not (name and ingredients and recipe_steps):
                                print(f"레시피 ID {recipe_id}에 본문 데이터가 없어 건너뜁니다.")
                                break

                            # 현재 CSV 파일 열기
                            with open(output_csv, mode='a', newline='', encoding='utf-8') as file:
                                fieldnames = ['id', 'name', 'ingredients', 'recipe', 'description', 'image', 'totalTime', 'recipeYield']
                                writer = csv.DictWriter(file, fieldnames=fieldnames)

                                if file.tell() == 0:
                                    writer.writeheader()

                                writer.writerow({
                                    'id': recipe_id,
                                    'name': name,
                                    'ingredients': ingredients,
                                    'recipe': ' '.join(recipe_steps),
                                    'image': image,
                                    'description': description,
                                    'totalTime': total_time,
                                    'recipeYield': recipe_yield,
                                })

                            print(f"레시피 ID {recipe_id}가 {output_csv}에 저장되었습니다.")
                            processed_ids.add(recipe_id)
                            current_count += 1

                            # 10,000개 도달 시 새 파일로 전환
                            if current_count >= 10000:
                                print(f"{output_csv} 저장 완료. 다음 파일로 전환합니다.")
                                current_file_index += 1
                                output_csv = f"{output_csv_prefix}_{current_file_index}.csv"
                                current_count = 0

                            break

                        except json.JSONDecodeError as e:
                            print(f"JSON 데이터 파싱 실패: {e}\n수정된 JSON:\n{fixed_json}")
                            break
                    else:
                        print(f"레시피 ID {recipe_id}: JSON 데이터가 포함된 script 태그를 찾을 수 없습니다.")
                        break
                else:
                    print(f"레시피 ID {recipe_id} 요청 실패 (상태 코드: {response.status_code})")
                    break

            except requests.RequestException as e:
                print(f"Error occurred while requesting {url}: {e}")
                attempt += 1
                if attempt < 5:
                    print(f"Retrying in 5 minutes... Attempt {attempt}/5")
                    time.sleep(300)
                else:
                    print("Max retries reached, skipping this recipe.")
                    break

    # 처리되지 않은 ID 업데이트
    remaining_ids = [recipe_id for recipe_id in recipe_ids if recipe_id not in processed_ids]
    with open(data_file, 'w', encoding='utf-8') as f:
        f.write('\n'.join(remaining_ids))
    print(f"{data_file} 업데이트 완료: {len(remaining_ids)}개의 ID 남음.")


In [4]:
# 사용
patterns = ["토마토", "방울 토마토", "김치", "가지", "오이", "애호박", "팽이버섯", "새송이 버섯",
    "돼지고기", "닭고기", "소고기", "두부", "콩나물", "대파", "양파", "마늘", "시금치",
    "고추", "깻잎", "당근", "감자", "고구마", "계란", "무", "파프리카", "맛살", "쌀",
    "어묵", "사과", "비엔나 소시지"]  # 검색할 재료 패턴 목록
recipe_id_file = "recipe_ids.txt"  # 레시피 ID가 저장된 파일
output_csv = "recipe_data_part"  # 저장할 CSV 파일

##fetch_recipes(patterns, recipe_id_file)
recipecsv(recipe_id_file, output_csv, 8)

레시피 ID 6915462에 본문 데이터가 없어 건너뜁니다.
레시피 ID 6915508에 본문 데이터가 없어 건너뜁니다.
JSON 데이터 파싱 실패: Invalid control character at: line 5 column 33 (char 100)
수정된 JSON:

{
    "@context": "http://schema.org/",
    "@type": "Recipe",
    "name": "감자황태국만들기#원기회복에 좋아요~
뽀얀국물이 사골국 못지 않아요.",
    "image": [
        "https://recipe1.ezmember.co.kr/cache/recipe/2019/07/05/5e6d87619201b2fae0457c6609944e831_f.jpg",
        "https://recipe1.ezmember.co.kr/cache/recipe/2019/07/05/5e6d87619201b2fae0457c6609944e831.jpg"
    ],
    "author": {
        "@type": "Person",
        "name": "캔캔맘"
    },
    "datePublished": "2019-07-09T07:57:07+09:00",
    "description": "딸래미가 몸살에 걸려 황태국 끓였어요. 요즘 감자도 포실포실 맛있구 감자 좋아하는 딸래미에게 딱이다 싶어 만들어 봤어요. 피로회복에도 도움을 준대요~"
        ,"aggregateRating": {
        "@type": "AggregateRating",
        "ratingValue": "5",
        "reviewCount": "3"
    }
            ,"totalTime": "PT0H60M"
            ,"recipeYield": "4 servings"
        ,"recipeIngredient": ["황태채 3종이컵","감자 3개","무우 1/4개","양파 1/2개",